In [1]:
import pandas as pd
import os

df = pd.read_csv('flair-vgg16-data.csv', names=['_id', 'message', 'image_concept', 'published', 'disabled'])
df['available'] = 0

all_images_path = 'data/all_images'
for i, row in df.iterrows():
    if os.path.isfile(os.path.join(all_images_path, row['_id'] + '.jpg')):
        df.at[i, 'available']= 1    
        
df_published = df.loc[df.query('available == 1 and published == 1').index]
df_published['text'] = df_published['image_concept'] + ' ' + df_published['message']
df_published = df_published.loc[df_published['text'].notnull()]

published_count = len(df_published)

df_disabled = df.loc[df.query('available == 1 and disabled == 1').index]
df_disabled['text'] = df_disabled['image_concept'] + ' ' + df_disabled['message']
df_disabled = df_disabled.loc[df_disabled['text'].notnull()]


df_disabled = df_disabled[:published_count]

print(f"published {len(df_published)}, disabled {len(df_disabled)}")

df_all = pd.concat([df_published, df_disabled], ignore_index=True)

df_all = df_all.reset_index(drop=True)

df_all

published 720, disabled 720


,_id,message,image_concept,published,disabled,available,text
0,5e5836fee917e8d9a8a7b277,endless blues greatbarrierreef australia whits...,seascape water shoal sea turquoise sun tropica...,1,0,1,seascape water shoal sea turquoise sun tropica...
1,5e58343ded065ad79e312f3d,hamiltonisland,tree travel vacation seashore water hotel isla...,1,0,1,tree travel vacation seashore water hotel isla...
2,5e57dc939e88b6be2ac42800,we are going coconuts for hamiltonisland here ...,relaxation beach sea vacation sand recreation ...,1,0,1,relaxation beach sea vacation sand recreation ...
3,5e55dca437fa5927dcdf02f3,en route to gbr embrace the elevation in luxur...,nature travel diving water sea underwater ocea...,1,0,1,nature travel diving water sea underwater ocea...
4,5e55d69eb9e5b725cd7ba02f,golf course views hamiltonislandgolfcourse whi...,outdoors landscape beach sky nature rural nope...,1,0,1,outdoors landscape beach sky nature rural nope...
...,...,...,...,...,...,...,...
1435,5e4e3124497f22be9069f067,golf trips with the boys are always wicked and...,sky water seashore sea travel winter ship land...,0,1,1,sky water seashore sea travel winter ship land...
1436,5e4e3124ffb21abead202386,golf trips with the boys are always wicked and...,travel golf ocean grass water sand nature sea ...,0,1,1,travel golf ocean grass water sand nature sea ...
1437,5e4e3124ffb21abead202385,throwback to that time i was warm and tanned q...,watercraft water people noperson recreation se...,0,1,1,watercraft water people noperson recreation se...
1438,5e4e3123164d73be9b8cd43e,golf trips with the boys are always wicked and...,adult people class girl grouptogether portrait...,0,1,1,adult people class girl grouptogether portrait...


In [2]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_all, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.4, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"train {len(train_df)}, val {len(val_df)}, test {len(test_df)}")

train 864, val 345, test 231


In [4]:
from flair.embeddings import (
    Sentence, 
    WordEmbeddings, 
    FlairEmbeddings, 
    StackedEmbeddings, 
    DocumentRNNEmbeddings,
    BytePairEmbeddings
)
from flair.training_utils import store_embeddings
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from torchvision import transforms, models
from PIL import Image
import os
import matplotlib.pyplot as plt

%matplotlib inline


import geffnet

model = models.densenet161(pretrained=True)
model

#model = geffnet.efficientnet_b2(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)

#print(model)
    
    
# for name, param in model.named_parameters():
#     print(name)
#model = nn.Sequential(*list(model.children())[:-1])
#print(model)

# model.classifier[1] = nn.Linear(1280, 1)



# model.eval()


#print(model(torch.zeros((1, 3,260,260))).shape)


Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /home/ec2-user/.cache/torch/checkpoints/densenet161-8d451a50.pth


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [5]:
train_on_gpu = torch.cuda.is_available()

    
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
#         self.document_embeddings = DocumentRNNEmbeddings([
#             WordEmbeddings('twitter'),
#         ], hidden_size=128)
        
        pretrained_model = models.densenet161(pretrained=True)
        for param in pretrained_model.parameters():
            param.requires_grad = False
                        
        pretrained_model.classifier = nn.Linear(2208, 1)
        self.pretrained_model = pretrained_model
                            
#         self.fc = nn.Linear(2048*10*10, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, sentences, images):
        batch_size = images.size(0)
        
#         self.document_embeddings.embed(sentences)
        
#         text_embedding_list = [
#             s.embedding.unsqueeze(0) for s in sentences
#         ]
        
#         text_embedding_tensor = torch.cat(text_embedding_list, 0).cuda()
        
#         if train_on_gpu:
#             images = images.cuda()
            
#         image_tensor = self.vgg16(images)
#         image_tensor = image_tensor.view(batch_size, -1)
        
#         input_tensor = torch.cat([text_embedding_tensor, image_tensor], 1)
        
        input_tensor = images.cuda()
        input_tensor = self.pretrained_model(input_tensor)
        #input_tensor = input_tensor.view(batch_size, -1)
        
        #print(input_tensor.shape)
        #out = self.sig(self.fc(input_tensor))
        out = self.sig(input_tensor)
        return out
    

inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
   std=[1/0.229, 1/0.224, 1/0.225]
)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

image_dimension = 260
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(image_dimension),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    normalize
])        

test_transform = transforms.Compose([
    transforms.Resize((image_dimension, image_dimension)),
    transforms.ToTensor(),
    normalize    
])

def get_batches(df, transformer, batch_size=16):
    n_batches = len(df)//batch_size    
    for i in range(0, len(df), batch_size):
        sentences = [Sentence(txt) for txt in df[i:i+batch_size]['message']]
        labels = [1 if label else 0 for label in df[i:i+batch_size]['published']]
        
        images = []
        for _id in df[i:i+batch_size]['_id']:
            image = Image.open('data/all_images/'+_id+'.jpg').convert('RGB')
            images.append(transformer(image).unsqueeze(0))
            
        image_tensor = torch.cat(images, 0)
        
        yield sentences, image_tensor, torch.FloatTensor(labels)
    


def visualise_data():
    fig = plt.figure(figsize=(15, 15))
    num_of_samples = 20
    num_cols = 5
    for i, (sentences, images, labels) in enumerate(get_batches(train_df, train_transform)):
        if i >= num_of_samples:
            break
        
        ax = fig.add_subplot(num_of_samples//num_cols + 1, num_cols, i+1, xticks=[], yticks=[])
        image = images[0]
        image = inv_normalize(image)
        plt.imshow(np.transpose(image, (1, 2, 0)))
        label = labels[0]
        ax.set_title(f"{label}")
        
        

def train_model(model, epochs, lr, train_df, val_df, checkpoint_file, early_stopping=20):        
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_loss = np.inf
    no_improvement = 0

    if train_on_gpu:
        model = model.cuda()
        
    for epoch in range(epochs):        
        total_train_loss = 0
        total_val_loss = 0
        train_loss = 0
        val_loss = 0
        
        # Train
        model.train()        
        for i, (sentences, images, labels) in enumerate(get_batches(train_df, train_transform)):         
            if train_on_gpu:
                labels = labels.cuda()
            
            optimizer.zero_grad()
            
            out = model(sentences, images)
            loss = criterion(out.squeeze(), labels)
            loss.backward()
            
            #nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
            
            total_train_loss += loss.item()
            
            store_embeddings(sentences, 'cpu')
            
            if i % 10 == 0:
                print(f"Epoch {epoch}, Batch {i}, train loss {loss.item()/labels.size(0)}")
            
            
        train_loss = total_train_loss/len(train_df)
        print(f"> Epoch {epoch}, train loss {train_loss}")
        
        # Eval
        model.eval()
        for sentences, images, labels in get_batches(val_df, test_transform):
            if train_on_gpu:
                labels = labels.cuda()
            
            out = model(sentences, images)
            loss = criterion(out.squeeze(), labels)
            total_val_loss += loss.item()
            
            store_embeddings(sentences, 'cpu')
            
            
        val_loss = total_val_loss / len(val_df)
        
        print(f"> Epoch {epoch}, val loss {val_loss}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            no_improvement = 0
            torch.save(model.state_dict(), checkpoint_file)
            print("Saved model.")
        else:
            no_improvement += 1
            print("No improvement.")
            if no_improvement >= early_stopping:
                print(f"Early Stopping")
                break
            
                                              
checkpoint_file = 'flair_vgg16_model_final.pt'      
lr = 0.0001
epochs = 20           

model = MyModel()
#print(model)
train_model(model, epochs, lr, train_df, val_df, checkpoint_file)

        


Epoch 0, Batch 0, train loss 0.04358367621898651
Epoch 0, Batch 10, train loss 0.04421008750796318
Epoch 0, Batch 20, train loss 0.04540610313415527
Epoch 0, Batch 30, train loss 0.045052625238895416
Epoch 0, Batch 40, train loss 0.041636206209659576
Epoch 0, Batch 50, train loss 0.04340512305498123
> Epoch 0, train loss 0.04335577192681807
> Epoch 0, val loss 0.0437370448872663
Saved model.
Epoch 1, Batch 0, train loss 0.04220261424779892
Epoch 1, Batch 10, train loss 0.04445686191320419
Epoch 1, Batch 20, train loss 0.04351738467812538
Epoch 1, Batch 30, train loss 0.043541133403778076
Epoch 1, Batch 40, train loss 0.03961005061864853
Epoch 1, Batch 50, train loss 0.04221518710255623
> Epoch 1, train loss 0.04246406843540845
> Epoch 1, val loss 0.04289913281150486
Saved model.
Epoch 2, Batch 0, train loss 0.040728483349084854
Epoch 2, Batch 10, train loss 0.0437706895172596
Epoch 2, Batch 20, train loss 0.04518665373325348
Epoch 2, Batch 30, train loss 0.04273947700858116
Epoch 2, Ba

In [6]:
def eval_model(model, test_df):
    if train_on_gpu:
        model = model.cuda()
        
    model.eval()
    num_correct = 0
    for i, (sentences, images, labels) in enumerate(get_batches(test_df, test_transform)):
        if train_on_gpu:
            labels = labels.cuda()
            
        out = model(sentences, images)
        pred = torch.round(out.squeeze())
        correct = (pred == labels)
        correct = correct.cpu().numpy() if train_on_gpu else correct.numpy()
        
        num_correct += np.sum(correct)
        
        store_embeddings(sentences, 'cpu')
        
    total = len(test_df)
    print(f"{num_correct}/{total} correct. Accuracy: {num_correct*100/total} %")
    
    
best_model = MyModel()
best_model.load_state_dict(torch.load(checkpoint_file))
eval_model(best_model, test_df)


159/231 correct. Accuracy: 68.83116883116882 %
